# 추천 시스템 (Recommender Systems)

* 추천 시스템은 크게 두가지로 구분 가능
  * 컨텐츠 기반 필터링 (content-based filtering)
  * 협업 필터링 (collaborative filtering)
* 두가지를 조합한 hybrid 방식도 가능
* 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
* 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 추천

## Surprise

* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 4.4MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670917 sha256=8b931c3e7c835d633d86a8ff6f929379d9d07cbce797f4074219e0ee164a0cb3
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


간단한 surprise 실습

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)
data.raw_ratings[:10]

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [ ]:
model = SVD()

In [ ]:
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9301  0.9401  0.9433  0.9380  0.9355  0.9374  0.0045  
MAE (testset)     0.7335  0.7420  0.7410  0.7419  0.7358  0.7388  0.0035  
Fit time          4.69    4.69    4.70    4.85    4.84    4.75    0.07    
Test time         0.14    0.19    0.25    0.14    0.20    0.18    0.04    


{'fit_time': (4.69245982170105,
  4.69375205039978,
  4.700906276702881,
  4.848453521728516,
  4.837285280227661),
 'test_mae': array([0.73350671, 0.7420353 , 0.74096088, 0.74191355, 0.73578283]),
 'test_rmse': array([0.93009461, 0.94005696, 0.94333874, 0.9380188 , 0.93554843]),
 'test_time': (0.13539528846740723,
  0.19060993194580078,
  0.2469189167022705,
  0.13663578033447266,
  0.20428895950317383)}

## 컨텐츠 기반 필터링 (Content-based Filtering)

* 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
  * ex) 내가 지금 까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
* 유사도를 기반으로 추천
* 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다.
  * 장점
    * 많은 수의 사용자를 대상으로 쉽게 확장 가능
    * 사용자가 관심을 갖지 않던 상품 추천 가능
  * 단점
    * 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
    * 사용자의 기존 관심사항을 기반으로만 추천 가능

In [ ]:
import numpy as np
from surprise import Dataset

* 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기
* 나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)

In [ ]:
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [ ]:
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])
shape = (n_users + 1, n_movies + 1)
shape

(943, 1682)

In [ ]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = 1.
adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    similarity = np.dot(my_vector, user_vector)
    if similarity > best_match:
      best_match = similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 183, Best Match ID: 275


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

* 유클리드 거리를 사용해 추천
$$euclidean = \sqrt{\sum_{d=1}^{D}(A_i - B_i)^2}$$
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if euclidean_dist < best_match:
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 14.832396974191326, Best Match ID: 737


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


* 코사인 유사도를 사용해 추천

\begin{equation}
cos \theta = \frac{A \cdot B}{||A|| \times ||B||}
\end{equation}
* 두 벡터가 이루고 있는 각을 계산

In [ ]:
def compute_cos_similarity(v1, v2):
  norm1 = np.sqrt(np.sum(np.square(v1)))
  norm2 = np.sqrt(np.sum(np.square(v2)))
  dot = np.dot(v1, v2)
  return dot / (norm1 * norm2)

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.5278586163659506, Best Match ID: 915


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [ ]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, mivie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating
adj_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if euclidean_dist < best_match:
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.0, Best Match ID: 12


In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 1.0, Best Match ID: 1


## 협업 필터링(Collaborative Filtering)

* 사용자와 항목의 유사성을 동시에 고려해 추천
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
  * 장점
    * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요 없다.
    * 기존의 관심사가 아니더라도 추천 가능
  * 단점
    * 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
    * 추가 특성을 사용하기 어려움

In [ ]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)

* KNN을 사용한 협업 필터링

In [ ]:
model = KNNBasic()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9817  0.9715  0.9794  0.9781  0.9863  0.9794  0.0048  
MAE (testset)     0.7737  0.7662  0.7726  0.7746  0.7794  0.7733  0.0042  
Fit time          0.48    0.70    1.01    1.32    0.80    0.86    0.29    
Test time         8.26    9.10    8.96    6.40    4.86    7.52    1.64    


{'fit_time': (0.47637295722961426,
  0.7016811370849609,
  1.011441946029663,
  1.3179571628570557,
  0.8008370399475098),
 'test_mae': array([0.77374443, 0.76623074, 0.77255479, 0.77464473, 0.7794348 ]),
 'test_rmse': array([0.98172805, 0.97151929, 0.97940637, 0.97811169, 0.9862784 ]),
 'test_time': (8.256501197814941,
  9.09681510925293,
  8.959142446517944,
  6.401399850845337,
  4.864153623580933)}

* SVD를 사용한 협업 필터링

In [ ]:
model = SVD()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9387  0.9371  0.9349  0.9362  0.9358  0.9366  0.0013  
MAE (testset)     0.7378  0.7406  0.7364  0.7385  0.7378  0.7382  0.0014  
Fit time          14.74   17.90   18.01   15.75   11.29   15.54   2.47    
Test time         0.69    0.54    0.44    0.25    0.15    0.42    0.19    


{'fit_time': (14.738423585891724,
  17.89631676673889,
  18.009915590286255,
  15.751114130020142,
  11.286134958267212),
 'test_mae': array([0.73783042, 0.7406177 , 0.73635095, 0.73852324, 0.7377922 ]),
 'test_rmse': array([0.93871097, 0.93714391, 0.93490378, 0.93618848, 0.93581977]),
 'test_time': (0.6900382041931152,
  0.5443565845489502,
  0.4352271556854248,
  0.25284647941589355,
  0.15283918380737305)}

* NMF를 사용한 협업 필터링

In [ ]:
model = NMF()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9719  0.9670  0.9627  0.9657  0.9588  0.9652  0.0044  
MAE (testset)     0.7663  0.7596  0.7578  0.7580  0.7536  0.7591  0.0041  
Fit time          13.33   16.04   15.89   13.66   9.72    13.72   2.29    
Test time         0.65    0.45    0.25    0.23    0.14    0.34    0.18    


{'fit_time': (13.32589864730835,
  16.03820824623108,
  15.88900899887085,
  13.655339002609253,
  9.716006994247437),
 'test_mae': array([0.76631714, 0.75960911, 0.7578202 , 0.75804749, 0.75363648]),
 'test_rmse': array([0.97192688, 0.9669696 , 0.96274285, 0.96570314, 0.95878131]),
 'test_time': (0.6467640399932861,
  0.45313596725463867,
  0.250443696975708,
  0.22652459144592285,
  0.13872790336608887)}

* SVD++를 사용한 협업 필터링

In [ ]:
model = SVDpp()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9172  0.9152  0.9221  0.9241  0.9224  0.9202  0.0034  
MAE (testset)     0.7188  0.7182  0.7213  0.7242  0.7272  0.7219  0.0034  
Fit time          626.02  617.67  614.90  621.30  154.19  526.82  186.35  
Test time         10.28   11.47   11.57   7.11    2.86    8.66    3.32    


{'fit_time': (626.0241029262543,
  617.6709825992584,
  614.8988263607025,
  621.3004302978516,
  154.190927028656),
 'test_mae': array([0.71875511, 0.71816116, 0.72132943, 0.72424938, 0.72717366]),
 'test_rmse': array([0.91724743, 0.91520464, 0.92210753, 0.92406634, 0.92238539]),
 'test_time': (10.276660680770874,
  11.471399784088135,
  11.571771144866943,
  7.107489109039307,
  2.861874580383301)}

## 하이브리드(Hybrid)

* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
* 많은 하이브리드 방식이 존재
* 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [ ]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)
raw_data[: 0] -= 1
raw_data[: 1] -= 1

In [ ]:
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])
shape = (n_users + 1, n_movies + 1)
shape

(944, 1683)

In [ ]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating

In [ ]:
adj_matrix

array([[                   0,             10273248,              2949721,
        ..., -8791026472627208172,     2322473542503828,
         5590240372031750144],
       [         69797871986,                    5,                    3,
        ...,    72705844289994752,               540743,
         4631672130349860352],
       [-8791026472627208172,                    4,  1966828965523881984,
        ...,  5118424077781958656,                   64,
         1459244349749302406],
       ...,
       [     139635215918192,                    5,      139635244327704,
        ...,      139635215991472,      139635215988872,
             139635718204640],
       [     139635215991536,      139635215988936,      139635718204696,
        ...,      139635215540528,      139635215537928,
             139635532589856],
       [     139635215540592,      139635215537992,                    5,
        ...,      139635215609712,      139635215611272,
             139635541984456]])

In [ ]:
U, S, V = randomized_svd(adj_matrix, n_components=2)
S = np.diag(S)

In [ ]:
print(U.shape)
print(S.shape)
print(V.shape)

(944, 2)
(2, 2)
(2, 1683)


In [ ]:
np.matmul(np.matmul(U, S), V)

array([[-6.44043543e+17, -4.04290863e+17,  7.15036329e+17, ...,
        -4.88056376e+17,  5.54957176e+16, -8.93660120e+16],
       [-1.68048106e+17, -1.76931608e+17,  3.08368869e+17, ...,
        -1.48444081e+17, -8.88378300e+16,  1.14335404e+16],
       [-3.53130840e+17, -2.41137513e+17,  4.25239124e+17, ...,
        -2.73350435e+17,  2.28011262e+15, -3.95318106e+16],
       ...,
       [ 1.06778540e+13,  1.35014289e+13, -2.34453507e+13, ...,
         1.00993386e+13,  8.91191491e+12, -1.82540207e+12],
       [ 9.56750303e+12,  1.27562427e+13, -2.21304726e+13, ...,
         9.24368747e+12,  8.93791895e+12, -1.95603726e+12],
       [ 1.18157407e+13,  1.23109936e+13, -2.14613851e+13, ...,
         1.03991480e+13,  6.05925406e+12, -7.40985672e+11]])

* 사용자 기반 추천
* 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
* 사용자 특징 벡터의 유사도 사용

In [ ]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999693033808538, Best Match ID: 131


In [ ]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[0, 3, 11, 20, 33, 40, 41, 42, 45, 51, 52, 63, 64, 74, 79, 81, 84, 93, 96, 101, 106, 112, 113, 118, 123, 125, 128, 135, 137, 140, 145, 152, 159, 164, 169, 176, 186, 191, 198, 205, 210, 222, 227, 229, 232, 239, 241, 244, 248, 255, 260, 265, 272, 274, 277, 289, 294, 299, 306, 323, 328, 340, 366, 379, 381, 384, 389, 403, 405, 408, 425, 432, 439, 458, 460, 463, 468, 485, 492, 497, 499, 502, 507, 512, 519, 526, 531, 533, 538, 543, 550, 557, 562, 569, 576, 593, 595, 600, 605, 607, 612, 614, 617, 624, 629, 634, 636, 639, 651, 658, 670, 675, 687, 692, 694, 697, 704, 711, 718, 723, 728, 740, 742, 747, 754, 756, 759, 778, 781, 793, 795, 802, 805, 811, 812, 817, 831, 833, 838, 845, 850, 857, 864, 869, 876, 878, 881, 887, 888, 893, 900, 914, 917, 926, 929, 936, 938, 941, 946, 960, 965, 972, 977, 984, 991, 998, 1003, 1029, 1034, 1041, 1058, 1063, 1070, 1075, 1082, 1087, 1094, 1101, 1106, 1108, 1111, 1118, 1134, 1146, 1151, 1156, 1161, 1173, 1178, 1180, 1190, 1197, 1204, 1209, 1221, 1226, 1233, 1240

* 항목 기반 추천
* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

In [ ]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_vector = -1, -1, []

for user_id, user_vector in enumerate(V.T):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999999954662253, Best Match ID: 1673


In [ ]:
recommend_list = []
for i, user_vector in enumerate(zip(adj_matrix)):
  if adj_matrix[i][my_id] > 0.9:
    recommend_list.append(i)
print(recommend_list)

[1, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 23, 24, 26, 27, 28, 29, 30, 31, 34, 35, 36, 38, 42, 43, 44, 45, 47, 49, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 72, 73, 75, 76, 77, 79, 82, 85, 86, 87, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 107, 108, 109, 110, 111, 113, 114, 115, 117, 119, 120, 122, 124, 125, 126, 127, 128, 129, 131, 132, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 152, 154, 156, 158, 159, 160, 161, 162, 163, 164, 165, 167, 168, 169, 170, 172, 173, 174, 175, 176, 177, 180, 181, 182, 185, 186, 187, 188, 189, 190, 191, 193, 194, 195, 196, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263

* 비음수 행렬 분해를 사용한 하이브리드 추천

In [ ]:
adj_matrix

array([[                   0,             10273248,              2949721,
        ..., -8791026472627208172,     2322473542503828,
         5590240372031750144],
       [         69797871986,                    5,                    3,
        ...,    72705844289994752,               540743,
         4631672130349860352],
       [-8791026472627208172,                    4,  1966828965523881984,
        ...,  5118424077781958656,                   64,
         1459244349749302406],
       ...,
       [     139635215918192,                    5,      139635244327704,
        ...,      139635215991472,      139635215988872,
             139635718204640],
       [     139635215991536,      139635215988936,      139635718204696,
        ...,      139635215540528,      139635215537928,
             139635532589856],
       [     139635215540592,      139635215537992,                    5,
        ...,      139635215609712,      139635215611272,
             139635541984456]])

In [ ]:
np.matmul

<ufunc 'matmul'>

* 사용자 기반 추천

In [ ]:
my_id, my_vector = 0, U[0]  # 내 id가 0번 이라고 가정
best_match, best_match_id, best_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.999999998824327, Best Match ID: 649


In [ ]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[]


* 항목 기반 추천

In [ ]:
my_id, my_vector = 0, V.T[0]  # 기준 항목이 0번 이라고 가정
best_match, best_match_id, best_match_vector = -1, -1, []

for item_id, item_vector in enumerate(V.T):
  if my_id != item_id:
    cos_similarity = compute_cos_similarity(my_vector, item_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = item_id
      best_match_vector = item_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999999011362093, Best Match ID: 489


In [ ]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
  if adj_matrix[i][my_id] > 0.9:
    recommend_list.append(i)
print(recommend_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 211, 212, 213, 214, 215, 217, 218, 219, 220, 222, 223, 224,